#### /!\/!\/!\ Attention les outputs de ce notebook sont en texte brut pour ne pas interférer avec le %run dans le notebook d'exploration /!\/!\/!\

In [1]:
# import des principales librairies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# paramétrage de la fonction display pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 50)

In [1]:
def presentation(df):
    """donne des détails du dataframe"""
    display(df.head(3))

    # calcul le % de NA
    na = df.isna()
    nanumber = np.count_nonzero(na == True)
    total_val = len(df.columns) * len(df)

    print(
        f'Pourcentage de NA dans le jeu de données : {nanumber/total_val*100:.2f} %'
    )
    print('df.shape :', df.shape)

In [10]:
def plot_na(df):
    """sortie de 2 graphiques : Drop des NA par ratio [ligne/colonne]"""
    x1 = []
    y1 = []

    x2 = []
    y2 = []

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    fig.suptitle('NA Drop', size=14, weight='bold')

    ax1.set_title('rows')
    ax2.set_title('columns')

    # AX1
    for i in range(0, 101, 10):
        x1.append(int(i * len(df.columns) / 100))
    for j in x1:
        y1.append(len(df.dropna(axis=0, thresh=j)))

    # AX2
    for i in range(0, 10, 1):
        x2.append(int(i * len(df) / 100))
    for j in x2:
        new_df = df.dropna(axis=1, thresh=j)
        y2.append(len(new_df.columns))

    ax1.plot(x1, y1, color='b', linestyle='dashed')
    ax2.plot(x2, y2, color='red', linestyle='dashed')

    ax1.set_xlabel('NA per rows')
    ax1.set_ylabel('total rows')

    ax2.set_xlabel('NA per columns')
    ax2.set_ylabel('total columns')

    plt.show()

In [14]:
def drop_col(df, na_ratio, drop_or_not):
    """création d'un nouveau df en fonction du ratio de NA choisi"""
    na_per_col = na_ratio * len(df)
    df_shape = df.dropna(axis=1, thresh=na_per_col).shape
    print('Drop des colonnes dont le nombre de valeurs réelles <',
          int(na_per_col), ': df.shape =', df_shape)

    if drop_or_not == True:
        df = df.dropna(axis=1, thresh=na_per_col, inplace=True)

In [15]:
def drop_row(df, na_ratio, drop_or_not):
    """création d'un nouveau df en fonction du ratio de NA choisi"""
    na_per_row = na_ratio * len(df.columns)
    df_shape = df.dropna(axis=0, thresh=na_per_row).shape
    print('Drop des lignes dont le nombre de valeurs réelles <',
          int(na_per_row), ': df.shape =', df_shape)

    if drop_or_not == True:
        df = df.dropna(axis=0, thresh=na_per_row, inplace=True)

In [9]:
def plot_min_max(df):
    """sortie de 2 graphiques : Classe les 20 premiers [min/max] par colonne"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    fig.suptitle('Les 10 indicateurs classés par :', size=14, weight='bold')

    ax1.set_title('Valeurs minimales')
    ax2.set_title('Valeurs maximales')

    # AX1
    y1 = df.iloc[2:, 2].sort_values().head(15)
    r1 = range(len(y1))

    # AX2
    y2 = df.iloc[2:, 3].sort_values(ascending=False).head(15)
    r2 = range(len(y2))

    ax1.barh(r1, y1, color='b', linestyle='dashed')
    ax2.barh(r2, y2, color='red', linestyle='dashed')

    ax1.set_yticks(r1)
    ax2.set_yticks(r2)

    ax1.set_yticklabels(y1.index)
    ax2.set_yticklabels(y2.index)

    plt.show()

In [10]:
def plot_stats(df):
    """sortie de 2 graphiques : Classe les [mean/median/std] par colonne"""
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))
    fig.suptitle('Les 10 indicateurs classés par :', size=14, weight='bold')

    ax1.set_title('Std')
    ax2.set_title('Mean')
    ax3.set_title('Median')

    # AX1
    y1 = df.iloc[2:, 0].sort_values(ascending=False).head(10)
    r1 = range(len(y1))

    # AX2
    y2 = df.iloc[2:, 1].sort_values(ascending=False).head(10)
    r2 = range(len(y2))

    # AX3
    y3 = df.iloc[2:, 4].sort_values(ascending=False).head(10)
    r3 = range(len(y2))

    ax1.bar(r1, y1, color='b', linestyle='dashed')
    ax2.bar(r2, y2, color='red', linestyle='dashed')
    ax3.bar(r3, y3, color='green', linestyle='dashed')

    ax1.set_xticks(r1)
    ax2.set_xticks(r2)
    ax3.set_xticks(r3)

    ax1.set_xticklabels(y1.index, rotation=90)
    ax2.set_xticklabels(y2.index, rotation=90)
    ax3.set_xticklabels(y3.index, rotation=90)

    plt.show()

In [11]:
def relevant_features(df):
    # enregistrement des indicateurs stats dans des dF spécifiques: dispersion et tendance centrale
    std_df = df.std(axis=0, numeric_only=True)
    mean_df = df.mean(axis=0, numeric_only=True)
    min_df = df.min(axis=0, numeric_only=True)
    max_df = df.max(axis=0, numeric_only=True)
    median_df = df.median(axis=0, numeric_only=True)

    # concaténation des 5 dF obtenues vers un dF unique
    df_feature = pd.concat([std_df, mean_df, min_df, max_df, median_df],
                           axis=1,
                           keys=['std', 'mean', 'min', 'max', 'median'])

    # supprime l'écritutre scientifique du dF
    df_feature = df_feature.astype(object)

    # appel des fonctions graphiques
    plot_min_max(df_feature)
    plot_stats(df_feature)

In [11]:
def plot_column(ser):
    """sortie de 2 graphiques : Un nuage de point et une box plot par pd.Series séléctionné en argument"""
    blood_diamond = dict(markerfacecolor='red', marker='D')

    fig = plt.figure(constrained_layout=True, figsize=(20, 7))
    gs = fig.add_gridspec(1, 4)

    fig.suptitle(f'Représentation graphique de la série : {ser.name}',
                 size=15,
                 weight='bold')

    ax1 = fig.add_subplot(gs[0, :-1])
    ax2 = fig.add_subplot(gs[0, -1:])

    ax2.tick_params(axis='both',
                    which='both',
                    bottom=False,
                    top=False,
                    left=False,
                    labelbottom=False,
                    labelleft=False)

    ax1.set_title('Nuage de points', size=14)
    ax2.set_title('Boxplot', size=14)

    ax1.scatter(ser.index, ser, c='red', marker='.')
    ax2.boxplot(ser.dropna(), vert=True, flierprops=blood_diamond)

    stat_txt = f"""
    n : {ser.count()}
    
    Med : {ser.median(axis=0, skipna=True):.2f}
    Mean : {ser.mean(axis=0, skipna=True):.2f}
    Std : {ser.std(axis=0, skipna=True):.2f}"""

    ax1.text(0.89, 0.78, stat_txt, transform=ax1.transAxes, fontsize=14)

In [6]:
def bar_column(ser):
    """sortie de 2 graphiques : Un nuage de point et une box plot par pd.Series séléctionné en argument"""
    blood_diamond = dict(markerfacecolor='red', marker='D')

    fig = plt.figure(figsize=(20, 7), constrained_layout=True)
    gs = fig.add_gridspec(4, 1)

    ax1 = fig.add_subplot(gs[0:3, :])
    ax2 = fig.add_subplot(gs[-1:, :])

    ax2.tick_params(axis='both',
                    which='both',
                    bottom=False,
                    left=False,
                    labelleft=False,
                    labelbottom=False)

    ax1.hist(ser.dropna(), bins='sturges', align='left')

    ax2.boxplot(ser.dropna(), vert=False, flierprops=blood_diamond)

    ax1.set_title(f'Représentation graphique de la série : {ser.name}',
                  size=20,
                  weight='bold')

    stat_txt = f"""
    n : {ser.count()}

    Mean : {ser.mean(axis=0, skipna=True):.2f}
    Std : {ser.std(axis=0, skipna=True):.2f}
  
    Min : {ser.min(axis=0, skipna=True):.2f}
    Median : {ser.median(axis=0, skipna=True):.2f}
    Max : {ser.max(axis=0, skipna=True):.2f}"""

    ax1.text(0.89, 0.6, stat_txt, transform=ax1.transAxes, fontsize=14)
    plt.savefig(ser.name)

In [5]:
def linear_regression(x, y):
    """Permets de réaliser l'analyse du jeu de donnée nettoyé"""
    # Affichage graphique
    plt.figure(figsize=(8, 5))

    # Titres et labels
    plt.title('Linear regression')
    plt.xlabel(x.name)
    plt.ylabel(y.name)

    # Reshape data
    x = np.array(x).reshape(-1, 1)
    y = np.array(y).reshape(-1, 1)

    # Split du jeu de données
    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
        x, y, test_size=0.3)
    # Création du modèle
    reg = linear_model.LinearRegression()
    # Entraînement du modèle
    reg.fit(xtrain, ytrain)
    # Prédiction
    ypred = reg.predict(xtest)

    # Affichage des principaux résultats statistiques de la régression
    print(f'MSE linear : {mean_squared_error(ytest, ypred):.5f}')
    print(f'Rsquare linear : {r2_score(ytest, ypred):.5f}')

    # Scatter et plot pour visualisation de la régression et du dataset prédit
    plt.scatter(xtest, ytest, marker='.', color='k', label='predicted dataset')
    plt.plot(xtest, ypred, color='red', label='regression')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    plt.show()